In [2]:
import scipy.io
import numpy as np
import tensorflow as tf

In [21]:
path = './pretrained-model/imagenet-vgg-verydeep-19'
vgg = scipy.io.loadmat(path)

In [22]:
vgg_layers = vgg['layers']
print(type(vgg_layers))

<class 'numpy.ndarray'>


In [23]:
for k, v in vgg.items():
    print('' + k + ': typeof ' + str(type(v)))

__globals__: typeof <class 'list'>
__version__: typeof <class 'str'>
__header__: typeof <class 'bytes'>
meta: typeof <class 'numpy.ndarray'>
layers: typeof <class 'numpy.ndarray'>


In [46]:
def _weights(layer, expected_layer_name):
    """
    Return the weights and bias from the VGG model for a given layer.
    """
    wb = vgg_layers[0][layer][0][0][2]
    W = wb[0][0]
    b = wb[0][1]
    layer_name = vgg_layers[0][layer][0][0][0][0]
    assert layer_name == expected_layer_name
    return W, b

In [44]:
print(vgg_layers[0][2][0][0][2][0][0].shape)

(3, 3, 64, 64)


In [45]:
def _relu(conv2d_layer):
    """
    Return the RELU function wrapped over a TensorFlow layer.Expects a Conv2d layer input.
    """
    return tf.nn.relu(conv2d_layer)

In [53]:
def _conv2d(prev_layer, layer, layer_name):
    """
    Return the Conv2D layer using the weights, biasses form the VGG model at 'layer'.
    """
    W, b = _weights(layer, layer_name)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b, (b.size)))
    return tf.nn.conv2d(prev_layer, filter=W, strides=[1, 1, 1, 1], padding="SAME") + b

In [48]:
def _conv2d_relu(prev_layer, layer, layer_name):
    """
    Return the Conv2D + RELU layer using the weights, biases for the VGG mode at 'layer'.
    """
    return _relu(_conv2d(prev_layer, layer, layer_name))

In [49]:
def _avgpool(prev_layer):
    """
    Return the AveragePooling layer.
    """
    return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [51]:
W, b = _weights(0, 'conv1_1')
print(W.shape, b.shape)

(3, 3, 3, 64) (64, 1)


In [72]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
COLOR_CHANNELS = 3
graph = {}
graph['input'] = tf.Variable(np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)), dtype='float32')
graph['conv1_1'] = _conv2d_relu(graph['input'], 0, 'conv1_1')
graph['conv1_2'] = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
graph['avgpool1'] = _avgpool(graph['conv1_2'])
graph['conv2_1'] = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
graph['conv2_2'] = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
graph['avgpool2'] = _avgpool(graph['conv2_2'])
graph['conv3_1'] = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
graph['conv3_2'] = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
graph['conv3_3'] = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
graph['conv3_4'] = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
graph['avgpool3'] = _avgpool(graph['conv3_4'])
graph['conv4_1'] = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
graph['conv4_2'] = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
graph['conv4_3'] = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
graph['conv4_4'] = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
graph['avgpool4'] = _avgpool(graph['conv4_4'])
graph['conv5_1'] = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
graph['conv5_2'] = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
graph['conv5_3'] = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
graph['conv5_4'] = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
graph['avgpool5'] = _avgpool(graph['conv5_4'])

In [73]:
X = np.random.randn(1,224, 224, 3)

In [74]:

sess = tf.Session()
sess.run(graph['input'].assign(X))

array([[[[ 0.39586627,  0.32448643,  0.9682584 ],
         [-0.42135197,  0.41118634,  0.0566809 ],
         [-0.1585692 , -1.2399156 ,  0.8725272 ],
         ...,
         [-1.3958759 ,  0.21274479, -3.0234625 ],
         [ 0.040133  , -1.5444252 ,  1.7151695 ],
         [-1.2164184 ,  0.35504553,  0.29384074]],

        [[-1.1439891 , -0.78632164,  1.7115105 ],
         [-0.97932094,  0.60873646, -0.5376923 ],
         [ 1.226451  ,  0.62510103,  0.13320167],
         ...,
         [-0.2558284 ,  0.8923493 , -1.389393  ],
         [ 0.03900012,  0.94842654,  0.975845  ],
         [ 0.97465694, -0.52570814, -0.04283566]],

        [[ 0.2268357 , -0.6347494 , -0.99195266],
         [ 0.1217519 ,  0.32183754, -0.54372495],
         [-0.44944593,  1.1058152 ,  1.0544038 ],
         ...,
         [ 0.7014382 ,  0.15918328, -0.38169593],
         [-0.73543835, -0.9335977 , -1.8758583 ],
         [ 0.80849713,  0.7221604 , -0.94129765]],

        ...,

        [[-1.1015191 , -1.2573376 , -0

In [75]:

out = graph['conv4_1']
a_C = sess.run(out)

In [76]:
print(a_C.shape)

(1, 28, 28, 512)


In [77]:
sess.close()